In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, sys
import tensorflow as tf

2023-06-18 01:52:16.178784: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 01:52:16.351358: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 01:52:16.352344: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 01:52:17.317846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import matplotlib.pyplot as plt
from tensorflow import keras

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy
from pathlib import Path

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.applications.imagenet_utils import preprocess_input

import warnings
warnings.filterwarnings("ignore")

# Set the size of the loaded images
IMG_SIZE_LOAD = 64

# Define the number of classes
NUM_CLASSES = 5

# Set a random seed for reproducibility
SEED = 77

# Define the number of training samples to use
TRAIN_NUM = 1000  # Use 1000 when you just want to explore a new idea, use -1 for full train


In [4]:
train_dir = '/media/amanpreet/Windows-SSD/Users/siaya/Downloads/resized_train/resized_train/' #training directory

In [5]:
# Specify the file path of the CSV file
p_resize = '/media/amanpreet/Windows-SSD/Users/siaya/Downloads/trainLabels.csv'

# Read the CSV file into a DataFrame
d_resize = pd.read_csv(p_resize)

# Display the first five rows of the DataFrame
d_resize.head(5)

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [6]:
def turn2DF(path):
    # Read the CSV file into a DataFrame
    dataframe_train = pd.read_csv(path)

    # Modify the values in the DataFrame
    for i in range(len(dataframe_train)):
        dataframe_train = dataframe_train.replace(dataframe_train.loc[i, 'image'], train_dir + str(dataframe_train.loc[i, 'image']) + '.jpeg')

    # Save the modified DataFrame to a new CSV file
    dataframe_train.to_csv('mydata.csv', index=False)  # index=False to exclude the DataFrame's index from the CSV file


In [7]:

def crop_image_from_gray(img, tol=7):
    # Check if the input image is grayscale
    if img.ndim == 2:
        # Create a binary mask based on the tolerance value
        mask = img > tol
        # Apply the mask to the image to crop out dark regions
        return img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim == 3:
        # Convert the RGB image to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # Create a binary mask based on the tolerance value
        mask = gray_img > tol

        # Check the shape of the resulting image after cropping
        check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if check_shape == 0:
            # If the resulting image is too dark and everything is cropped out,
            # return the original image
            return img
        else:
            # Crop each channel of the RGB image based on the mask
            img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
            img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
            img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]

            # Stack the cropped channels back together
            img = np.stack([img1, img2, img3], axis=-1)
        return img

def load_ben_color(path, sigmaX=10):
    # Load the image using OpenCV
    image = cv2.imread(path)
    # Convert the image to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Crop the image to remove dark regions
    image = crop_image_from_gray(image)
    # Resize the image to a specified size
    image = cv2.resize(image, (IMG_SIZE_LOAD, IMG_SIZE_LOAD))
    # Apply a weighted combination of the image and its Gaussian blur to enhance details
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), sigmaX), -4, 128)

    return image


In [10]:
from keras.applications.imagenet_utils import preprocess_input
import numpy as np

def arr_img():
    # Create an array to store the image data
    xdata = np.zeros((d_resize.shape[0], IMG_SIZE_LOAD, IMG_SIZE_LOAD, 3))

    # Iterate over the rows of the DataFrame
    for i in range(len(d_resize)):
        # Construct the path to the image
        path = train_dir + str(d_resize.loc[i, 'image']) + '.jpeg'

        # Read and resize the image using OpenCV
        image = cv2.imread(path)
        image = cv2.resize(image, (IMG_SIZE_LOAD, IMG_SIZE_LOAD))

        # Convert the image to a NumPy array
        x = tf.keras.preprocessing.image.img_to_array(image)

        # Store the image data in the array
        xdata[i] = x

    # Normalize the image data by dividing by 255.0
    xdata = xdata / 255.0

    return xdata


In [ ]:
x_train = arr_img()

# Check the shape of the processed image data
print(x_train.shape)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

# Create an instance of LabelEncoder
lb = LabelEncoder()

# Encode the labels in the 'level' column of the DataFrame
y = lb.fit_transform(d_resize['level'])

# Convert the encoded labels to one-hot encoded format
y = to_categorical(y, num_classes=NUM_CLASSES)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

# Create a Sequential model
model = Sequential()

# Add a Conv2D layer with 30 filters, kernel size (5, 5), input shape (64, 64, 3), and ReLU activation
model.add(Conv2D(filters=30, kernel_size=(5, 5), input_shape=(64, 64, 3), activation='relu'))

# Add a MaxPooling2D layer with pool size (2, 2)
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add a Dropout layer with a rate of 0.2
model.add(Dropout(0.2))

# Add another Conv2D layer with 15 filters and kernel size (3, 3), followed by ReLU activation
model.add(Conv2D(filters=15, kernel_size=(3, 3), activation='relu'))

# Add another Conv2D layer with 15 filters and kernel size (3, 3), followed by ReLU activation
model.add(Conv2D(filters=15, kernel_size=(3, 3), activation='relu'))

# Add another MaxPooling2D layer with pool size (2, 2)
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another Dropout layer with a rate of 0.2
model.add(Dropout(0.2))

# Flatten the previous layer's output
model.add(Flatten())

# Add a Dense layer with 128 units and ReLU activation
model.add(Dense(128, activation='relu'))

# Add another Dense layer with 64 units and ReLU activation
model.add(Dense(64, activation='relu'))

# Add another Dense layer with 32 units and ReLU activation
model.add(Dense(32, activation='relu'))

# Add the final Dense layer with 5 units for multi-class classification and softmax activation
model.add(Dense(5, activation='softmax'))

# Compile the model with categorical cross-entropy loss, Adam optimizer, and metrics including categorical accuracy
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
